In [194]:
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import sklearn.cross_validation as cv
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from DecisionTree import DecisionTree
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import GradientBoostingRegressor
from GradientBoosting import GradientBoosting as myGB
# Plotting config
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [99]:
FOLDER = "data/"
FILES = [
        "iris.txt", "bezdekIris.txt", "wine.txt", "bupa.txt", "housing.txt", "spam"
        ]
FILE = "housing.txt"

In [100]:
# Подготавливаем признаки и целевую функцию
if FILE in FILES[:5]:
    
    df = pd.read_csv(FOLDER+FILE, sep=",", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    
    if FILE in FILES[:2]:
        # ИРИСЫ
        df[4] = pd.factorize(df[4])[0]
        X = df[[0,1,2,3]].as_matrix()
        y = df[4].as_matrix()

    if FILE == FILES[2]:
        x_indexes = [x for x in range(1,14)]
        X = df[x_indexes].as_matrix()
        y = df[0].as_matrix()

    if FILE == FILES[3]:
        X = df[[0,1,2,3,4,5]].as_matrix()
        y = df[6]
        
    if FILE == FILES[4]:
        df = pd.read_csv(FOLDER+FILE, sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
        X = df[df.columns[1:]].as_matrix()
        y = df[df.columns[0]].as_matrix()
        
    x_train, x_test, y_train, y_test = cv.train_test_split(X, y, test_size=0.25)

else:
    
    df_train = pd.read_csv(FOLDER+FILE+".train.txt", sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    df_test = pd.read_csv(FOLDER+FILE+".test.txt", sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    x_train = df_train[df_train.columns[1:]].as_matrix()
    y_train = df_train[df_train.columns[0]].as_matrix()
    x_test = df_test[df_test.columns[1:]].as_matrix()
    y_test = df_test[df_test.columns[0]].as_matrix()

# Дерево решений

### Классификация

In [81]:
%%time
for i in range(10):
    my_tree = DecisionTree(max_features=20, min_samples_leaf=5, max_steps=100)
    my_tree.fit(x_train, y_train)
    y_predicted = my_tree.predict(x_test)
    print accuracy_score(y_test, y_predicted)

0.763524264121
0.564836913286
0.67054494829
0.71469769292
0.641507557677
0.517800318218
0.731404136834
0.650258552108
0.626988862371
0.66099840891
CPU times: user 40 s, sys: 0 ns, total: 40 s
Wall time: 40 s


In [101]:
%%time
sklearn_tree = DecisionTreeClassifier(splitter='random')
sklearn_tree.fit(x_train, y_train)
y_predicted = sklearn_tree.predict(x_test)
print accuracy_score(y_test, y_predicted)

ValueError: Unknown label type: array([[  7.89600000e-02],
       [  7.90410000e-01],
       [  5.69175000e+00],
       [  1.44383000e+01],
       [  3.83518000e+01],
       [  9.92485000e+00],
       [  2.29270000e-01],
       [  1.68118000e+01],
       [  1.75050000e-01],
       [  3.96100000e-02],
       [  1.43900000e-02],
       [  4.89822000e+00],
       [  9.88430000e-01],
       [  3.87350000e-01],
       [  2.07162000e+01],
       [  6.53876000e+00],
       [  1.30100000e-02],
       [  8.25260000e-01],
       [  8.66400000e-02],
       [  1.64390000e-01],
       [  8.05579000e+00],
       [  4.35710000e-01],
       [  1.11081000e+01],
       [  7.72990000e-01],
       [  2.81838000e+00],
       [  5.82401000e+00],
       [  4.41700000e-02],
       [  8.24400000e-02],
       [  1.31580000e-01],
       [  7.35341000e+01],
       [  4.52700000e-02],
       [  1.73310000e-01],
       [  9.33889000e+00],
       [  1.25790000e-01],
       [  1.39600000e-01],
       [  2.73100000e-0

### Регрессия

In [192]:
my_tree = DecisionTree(is_classification=False, max_features=len(x_train[0]), max_steps=None)
my_tree.fit(x_train, y_train)
y_predicted = my_tree.predict(x_test)
print mse(y_test, y_predicted)

96.0974456896


In [185]:
from sklearn.tree import DecisionTreeRegressor

sklearn_tree = DecisionTreeRegressor(splitter='random')
sklearn_tree.fit(x_train, y_train)
y_predicted = sklearn_tree.predict(x_test)
print mse(y_test, y_predicted)

98.3203884113


# Градиентный бустинг

In [ ]:
my_gb = myGB(n_estimators=10, shrinkage=0.25, max_steps=None)
my_gb.fit(x_train, y_train)
y_predicted = my_gb.predict(x_test)
print mse(y_test, y_predicted)

In [256]:
gb = GradientBoostingRegressor(n_estimators=10, random_state=np.random)
gb.fit(x_train, y_train)
y_predicted = gb.predict(x_test)
print mse(y_test, y_predicted)

75.5023271785
